In [1]:
"""
Created on Wed Jul  7 17:46:12 2021

@author: Schroeder
"""

from turtle import color
import matplotlib
from matplotlib import colors

from IPython.display import display, HTML

from pathlib import Path
import csv

from datetime import datetime
from datetime import date
import calendar

import os
from os import listdir
from os.path import isfile, join

import sys, getopt

# Imports
#from pandas_datareader import data as pdr
#from yahoo_fin import stock_info as si

import numpy as np
from sklearn import linear_model
import scipy.stats


import numpy as np
from sklearn import linear_model

import pandas as pd

import plotly.io as pio
import plotly.express as px
from plotly.figure_factory import create_candlestick
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot

import seaborn as sns
from IPython.display import display, HTML

from pandas import ExcelWriter

# from sympy import maximum

import RSI_strat_SETUP

#init_notebook_mode()

hier:  C:\Michael\TRADING\ETFS\Analyse_data


In [12]:
def main(fad, stock):


    #onlyfiles = [f for f in listdir(mypath) if (isfile(join(mypath, f)) and not f.startswith("_")) ]

    b=[]
    c=[]
    figs=[]
    counter  = 0

    keys = ["Sector","Industry"]
    a=[]    
    b=[]
    
    for key in keys:
        'Baue Dataframe aus der exisitierendnen Res Datei. Relevnt sind die Sektoren und die 5d,10,20d,... returns'
        df = pd.read_csv(fad +"\\"+stock,sep=";",decimal=',', index_col=0)
        
        ###### Remove all Rows with bullshit Entrys such as NaN, etc. 
        df = df.dropna()
        discard = ["sector","ETF","Industry","nan"]
        df = df[~df.Sector.str.contains('|'.join(discard))]
        df = df[~df.Industry.str.contains('|'.join(discard))]
        
        csa = [key]
        csa = csa +  df.columns[:5].tolist()
        sectorsOrIndustry_analysis = pd.DataFrame(columns=csa)

        ## liste aller Sektor bzw. Industry Bezeichner
        list_of_sectorsOrIndustry = df[key].unique().tolist()
        sectorsOrIndustry_analysis[key]=df[key].unique().tolist()


        #print(key,": ",sectorsOrIndustry_analysis[key])

        # try:
        #     list_of_sectorsOrIndustry=list_of_sectorsOrIndustry.remove("nan")
        #     list_of_sectorsOrIndustry=list_of_sectorsOrIndustry.remove("sector")
        #     list_of_sectorsOrIndustry=list_of_sectorsOrIndustry.remove("ETF")
        #     list_of_sectorsOrIndustry=list_of_sectorsOrIndustry.remove("Industry")

        #     sectorsOrIndustry_analysis=sectorsOrIndustry_analysis.remove("nan")
        #     sectorsOrIndustry_analysis=sectorsOrIndustry_analysis.remove("sector")
        #     sectorsOrIndustry_analysis=sectorsOrIndustry_analysis.remove("ETF")
        #     sectorsOrIndustry_analysis=list_of_sectorsOrIndustry.remove("Industry")
        # except:
        #     print("-->")
            

        # Wenn nach Industry aufgelöst wird, dann fuege den Sektor in den TAble dazu 
        if key == "Industry":
            for i in sectorsOrIndustry_analysis[key]:
                try:
                    a=df["Sector"][df["Industry"]==i]
                    b.append(a[0])
                except:
                    print("errours:",key,", ",i)

            #print("b: ",b)    
            sectorsOrIndustry_analysis["Sector"]=b
        ################################

        for dt in df.columns[:5].tolist():
            yields=[]
            for sec in sectorsOrIndustry_analysis[key]:
                yields.append(df.loc[df[key]==sec, dt].mean())
            #print(yields)
            sectorsOrIndustry_analysis[dt]=yields

        
        #cm2 = sns.light_palette("red", as_cmap=True)
        cm1 = sns.blend_palette(['red',"white", 'green'], as_cmap=True)
        #display()

        sectorsOrIndustry_analysis=sectorsOrIndustry_analysis.sort_values(by="Sector",ascending=False)
        
        display(sectorsOrIndustry_analysis.style.set_precision(2).background_gradient(cmap=cm1,axis=0)       )

        f = open(fad+key+"_s.html",'w'); f.write(sectorsOrIndustry_analysis.style.set_precision(2).set_caption('Spaltenweise Color').background_gradient(axis=0,cmap=cm1).render()); f.close()
        f = open(fad+key+"_z.html",'w'); f.write(sectorsOrIndustry_analysis.style.set_precision(2).set_caption('Zeilenweise Color').background_gradient(axis=1,cmap=cm1).render()); f.close()    
        
    

In [3]:
currentdir = os.path.abspath('')
parentdir = os.path.realpath(os.path.join(currentdir, '..'))
sys.path.insert(0, parentdir) 
#############################################################

pfad = RSI_strat_SETUP.pfad

from MomentumScreening import my_setup
###
### The whole /ETFS/ Tree has to be located on ame level as repository !
pfad = os.path.realpath(os.path.join(parentdir, 'ETFS'))


### --------------------------------------------------------------------------
### Welcher Sektor oder index soll analysieert werden ?

In [4]:
who = "SP500"

In [5]:

who = "RES_" + who 
pfad = os.path.realpath(os.path.join(pfad, who))
pfad = os.path.realpath(os.path.join(pfad, 'Res'))


### ---------------------------------------------------------------------

In [13]:
#main(stock,          back_gap, fwd_gap, threshold rendite backward, threshold renidte forward,comment )
file = "SP502022-10-15__2022-06-23.csv"
main(pfad, file)

,Sector,80d,50d,21d,10d,5d
5,Utilities,-7.58,-13.45,-12.00,-2.47,-1.84
3,Technology,-10.33,-15.50,-9.12,-2.17,-4.56
1,Real Estate,-15.78,-19.59,-14.13,-1.38,-0.04
2,Industrials,-3.20,-11.06,-5.29,2.54,-1.23
4,Healthcare,-11.42,-13.46,-8.65,-1.06,-2.17
0,Energy,9.36,4.54,-1.11,8.14,-2.07
6,Consumer Defensive,-6.59,-9.97,-5.56,0.93,0.18
8,Consumer Cyclical,-10.93,-14.36,-8.38,1.92,-1.19
9,Communication Services,-15.82,-16.78,-11.03,-0.93,-3.06
7,Basic Materials,-9.83,-11.92,-4.31,0.59,-2.82


,Industry,80d,50d,21d,10d,5d,Sector
17,Utilities—Regulated Water,5.21,-4.17,-1.79,6.24,1.51,Utilities
117,Utilities—Renewable,-26.51,-29.97,-21.49,-8.08,-6.14,Utilities
8,Utilities—Regulated Electric,-6.06,-13.23,-12.53,-3.86,-2.15,Utilities
92,Utilities—Regulated Gas,-12.30,-12.56,-11.09,0.34,0.51,Utilities
55,Utilities—Independent Power Producers,-4.12,-9.94,-12.18,-0.89,-3.54,Utilities
41,Utilities—Diversified,-7.27,-12.86,-11.15,-2.05,-1.66,Utilities
64,Scientific & Technical Instruments,3.14,-8.25,-5.38,0.91,-2.23,Technology
127,Semiconductor Equipment & Materials,-24.54,-21.40,-10.04,-4.45,0.75,Technology
126,Consumer Electronics,-22.86,-23.63,-12.40,12.50,-2.75,Technology
123,Electronics & Computer Distribution,-14.46,-19.12,-5.53,1.57,-1.22,Technology
